In [ ]:
import tensorflow as tf

config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
tf.compat.v1.Session(config=config)

# gpus = tf.config.list_physical_devices('GPU')
# for i in range(len(gpus)): # gogo 나도 그거 찾움
	# tf.config.experimental.set_memory_growth(gpus[i], True)
# 오타 수정해써 끝ㅏㅜ 눌러바 gpu -> gpus 👍 초기화 하고 다시~

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import keras.backend as K
from keras.layers import Dense, Dropout
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.optimizers import SGD

from shutil import copy
from collections import defaultdict
import os

# Function to prepare training set and test set
def prepare_data(filepath, src, dest):
    classes_images = defaultdict(list)
    with open(filepath, 'r') as txt:
        paths = [read.strip() for read in txt.readlines()]
        for p in paths:
            food = p.split('/')
            classes_images[food[0]].append(food[1] + '.jpg')

    for food in classes_images.keys():
        print("\nCopying images into", food)
        if not os.path.exists(os.path.join(dest, food)):
            os.makedirs(os.path.join(dest, food))
        for i in classes_images[food]:
            copy(os.path.join(src, food, i), os.path.join(dest, food, i))
    print("Copying Done!")

# prepare_data('food-101/meta/train.txt', 'food-101/images', 'food-101/train')
# prepare_data('food-101/meta/test.txt', 'food-101/images', 'food-101/test')

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, Input, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

def identity_block(X, filters, kernel_size):
    X_shortcut = X

    X = Conv2D(filters, kernel_size, padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)

    X = Conv2D(filters, kernel_size, padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)

    X = Conv2D(filters, kernel_size, padding='same')(X)
    X = BatchNormalization()(X)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

def convolutional_block(X, filters, kernel_size):
    X_shortcut = X

    X = Conv2D(filters, kernel_size, padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)

    X = Conv2D(filters, kernel_size, padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)

    X = Conv2D(filters, kernel_size, padding='same')(X)
    X = BatchNormalization()(X)

    X_shortcut = Conv2D(filters, kernel_size, padding='same')(X_shortcut)
    X_shortcut = BatchNormalization()(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

def ResNet20(input_shape=(32, 32, 3), classes=101):
    X_input = Input(input_shape)
    X = X_input

    X = convolutional_block(X, 64, (3,3))
    X = identity_block(X, 64, (3,3))
    X = MaxPooling2D(2, 2, padding='same')(X)

    X = convolutional_block(X, 128, (3,3))
    X = identity_block(X, 128, (3,3))
    X = MaxPooling2D(2, 2, padding='same')(X)

    X = convolutional_block(X, 256, (3,3))
    X = identity_block(X, 256, (3,3))
    X = MaxPooling2D(2, 2, padding='same')(X)

    X = convolutional_block(X, 512, (3,3))
    X = identity_block(X, 512, (3,3))
    X = MaxPooling2D(2, 2, padding='same')(X)

    X = GlobalAveragePooling2D()(X)
    X = Dense(classes, activation='softmax')(X)

    model = Model(inputs=X_input, outputs=X, name='ResNet20')

    return model

K.clear_session()

n_classes = 101
img_width, img_height = 32, 32
train_data_dir = 'food-101/train'
validation_data_dir = 'food-101/test'
nb_train_samples = 75750
nb_validation_samples = 25250
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

model = ResNet20(input_shape=(img_height, img_width, 3), classes=n_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='best_model_resnet20.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('history_resnet20.log')

history = model.fit_generator(train_generator,
                              steps_per_epoch=nb_train_samples // batch_size,
                              validation_data=validation_generator,
                              validation_steps=nb_validation_samples // batch_size,
                              epochs=20,
                              verbose=1,
                              callbacks=[csv_logger, checkpointer])

model.save('model_trained_resnet20.h5')


In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import keras.backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, CSVLogger

from shutil import copy
from collections import defaultdict

# Function to prepare training set and test set
def prepare_data(filepath, src, dest):
    classes_images = defaultdict(list)
    with open(filepath, 'r') as txt:
        paths = [read.strip() for read in txt.readlines()]
        for p in paths:
            food = p.split('/')
            classes_images[food[0]].append(food[1] + '.jpg')

    for food in classes_images.keys():
        print("\nCopying images into", food)
        if not os.path.exists(os.path.join(dest, food)):
            os.makedirs(os.path.join(dest, food))
        for i in classes_images[food]:
            copy(os.path.join(src, food, i), os.path.join(dest, food, i))
    print("Copying Done!")

# prepare_data('food-101/meta/train.txt', 'food-101/images', 'food-101/train')
# prepare_data('food-101/meta/test.txt', 'food-101/images', 'food-101/test')

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, Input, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

def identity_block(X, filters, kernel_size):
    X_shortcut = X

    X = Conv2D(filters, kernel_size, padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)

    X = Conv2D(filters, kernel_size, padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)

    X = Conv2D(filters, kernel_size, padding='same')(X)
    X = BatchNormalization()(X)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

def convolutional_block(X, filters, kernel_size):
    X_shortcut = X

    X = Conv2D(filters, kernel_size, padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)

    X = Conv2D(filters, kernel_size, padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)

    X = Conv2D(filters, kernel_size, padding='same')(X)
    X = BatchNormalization()(X)

    X_shortcut = Conv2D(filters, kernel_size, padding='same')(X_shortcut)
    X_shortcut = BatchNormalization()(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

def ResNet20(input_shape=(299, 299, 3), classes=101):
    X_input = Input(input_shape)
    X = X_input

    X = convolutional_block(X, 64, (3,3))
    X = identity_block(X, 64, (3,3))
    X = MaxPooling2D(2, 2, padding='same')(X)

    X = convolutional_block(X, 128, (3,3))
    X = identity_block(X, 128, (3,3))
    X = MaxPooling2D(2, 2, padding='same')(X)

    X = convolutional_block(X, 256, (3,3))
    X = identity_block(X, 256, (3,3))
    X = MaxPooling2D(2, 2, padding='same')(X)

    X = convolutional_block(X, 512, (3,3))
    X = identity_block(X, 512, (3,3))
    X = MaxPooling2D(2, 2, padding='same')(X)

    X = GlobalAveragePooling2D()(X)
    X = Dense(classes, activation='softmax')(X)

    model = Model(inputs=X_input, outputs=X, name='ResNet20')

    return model

K.clear_session()

n_classes = 101
img_width, img_height = 299, 299
train_data_dir = 'food-101/train'
validation_data_dir = 'food-101/test'
nb_train_samples = 75750
nb_validation_samples = 25250
batch_size = 4

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

model = ResNet20(input_shape=(img_height, img_width, 3), classes=n_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

checkpointer = ModelCheckpoint(filepath='best_model_resnet20.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('history_resnet20.log')

history = model.fit_generator(train_generator,
                              steps_per_epoch=nb_train_samples // batch_size,
                              validation_data=validation_generator,
                              validation_steps=nb_validation_samples // batch_size,
                              epochs=10,
                              verbose=1,
                              callbacks=[csv_logger, checkpointer])

model.save('model_trained_resnet20.h5')
